<a href="https://colab.research.google.com/github/szeshiang/Advanced-Computer-Vision/blob/main/070624_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
# import the library
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,AveragePooling2D,Flatten,Dense,MaxPool2D, BatchNormalization,Dropout
from tensorflow.keras.datasets import mnist
print('library imported')

library imported


In [77]:
# load and preprocess the mnist dataset
(X_train,y_train),(X_test,y_test)=mnist.load_data()

In [78]:
X_train.shape

(60000, 28, 28)

In [79]:
X_train=X_train.reshape(-1,28,28,1).astype('float32')/255.0
X_test=X_test.reshape(-1,28,28,1).astype('float32')/255.0

In [80]:
X_train.shape

(60000, 28, 28, 1)

In [81]:
y_train[0:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [82]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [83]:
y_train[0:5]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [86]:
# Define the AlexNet-like model for MNIST
def alexnet_mnist():
    model=Sequential()


    # Conv1
    model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))


    # Conv2
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))


    # Conv3
    model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))


    # Conv4
    model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))


    # Conv5
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))


    # Flatten and Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))


    # Output Layer
    model.add(Dense(10, activation='softmax'))

    return model

In [87]:
# Call the function to create the model
model = alexnet_mnist()
model.summary() # Moved outside the function

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 96)        960       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 256)       221440    
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 256)       1024      
 chNormalization)                                                
                                                        

In [88]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
print('model is compiled')

model is compiled


In [90]:
def lr_schedule(epoch,lr):
  if epoch<5:
    return lr
  else:
    return lr*tf.math.exp(-0.1)

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
lr_scheduler=LearningRateScheduler(lr_schedule)


In [91]:
# Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rotation_range=10,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1)

In [92]:
# use modelcheckpoint
checkpoint_callback=ModelCheckpoint(filepath='alexnet_model.h5',
                                    monitor='val_accuracy',
                                    save_best_only=True,
                                    mode='max',
                                    save_freq='epoch')

In [94]:
# Train the model

history=model.fit(datagen.flow(X_train,y_train,batch_size=64),epochs=5,
                  validation_data=(X_test,y_test),
                  callbacks=[lr_scheduler,checkpoint_callback])

Epoch 1/5
938/938 [==============================] - ETA: 0s - loss: 0.2803 - accuracy: 0.9179

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - 46s 40ms/step - loss: 0.2803 - accuracy: 0.9179 - val_loss: 0.0999 - val_accuracy: 0.9696 - lr: 0.0010
Epoch 2/5
938/938 [==============================] - 33s 35ms/step - loss: 0.1206 - accuracy: 0.9686 - val_loss: 0.0936 - val_accuracy: 0.9744 - lr: 0.0010
Epoch 3/5
938/938 [==============================] - 33s 35ms/step - loss: 0.1096 - accuracy: 0.9730 - val_loss: 0.0713 - val_accuracy: 0.9819 - lr: 0.0010
Epoch 4/5
938/938 [==============================] - 33s 35ms/step - loss: 0.1003 - accuracy: 0.9760 - val_loss: 0.0643 - val_accuracy: 0.9849 - lr: 0.0010
Epoch 5/5
938/938 [==============================] - 30s 32ms/step - loss: 0.0918 - accuracy: 0.9782 - val_loss: 0.0868 - val_accuracy: 0.9811 - lr: 0.0010


In [95]:
# evaluate the model
test_loss,test_acc=model.evaluate(X_test,y_test)
print('Test Accuracy',test_acc)
print('Test Loss',test_loss)

313/313 [==============================] - 2s 6ms/step - loss: 0.0868 - accuracy: 0.9811
Test Accuracy 0.9811000227928162
Test Loss 0.08675733953714371


In [98]:
# load the model
import tensorflow as tf
best_model=tf.keras.models.load_model('/content/alexnet_model.h5')
test_loss,test_acc=best_model.evaluate(X_test,y_test)
print('Test Accuracy',test_acc)
print('Test Loss',test_loss)

313/313 [==============================] - 2s 5ms/step - loss: 0.0643 - accuracy: 0.9849
Test Accuracy 0.9848999977111816
Test Loss 0.0642935112118721
